# MPIA Arxiv on Deck 2: reprocessing

This notebook reprocesses some papers with the current baseline.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
import re
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper to reprocess

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname

In [4]:
from glob import glob
paper_refs = [k.split('/')[-1].replace('.md',  '') for k in glob('_build/html/**md')]
n_papers = len(paper_refs)
print(f"Found {n_papers:,d} from past processing.")

Found 172 from past processing.


In [5]:
new_papers = [get_paper_from_identifier(k) for k in tqdm(paper_refs)]

# select only papers with matching author names and highlight authors
candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(re.sub('\s*?-\s*?', '-', k)) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

  0%|          | 0/172 [00:00<?, ?it/s]

Arxiv has 172 new papers today
          172 with possible author matches


In [6]:
# move papers that are not matching

import os
import shutil

storage = "_build/html"

try:
    os.makedirs(os.path.join(storage, "removed"))
except FileExistsError:
    pass

for paper in new_papers:
    if paper not in candidates:
        print(f"moving {paper['identifier']} to {storage}/removed")
        try:
            shutil.move(os.path.join(storage, paper['identifier'] + '.md'),
                        os.path.join(storage, "removed", paper['identifier'] + '.md'))
            shutil.move(os.path.join(storage, "tmp_" + paper['identifier']),
                        os.path.join(storage, "removed", "tmp_" + paper['identifier']))
        except:
            pass

# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [ ]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/171 [00:00<?, ?it/s]

/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2203.15822/_region_.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure figure/figure1
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure figure/figure1 as tmp_2203.15822/./figure/figure1.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.

Found 89 bibliographic references in tmp_2203.15822/ms_fcc167.bbl.


/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2204.00342/MagneticEnergy.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:389: LatexWarning: Latex injecting: 'TableB1' from 'tmp_2204.00342/TableB1.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure P_M.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/

Found 114 bibliographic references in tmp_2204.00342/MagneticEnergy.bbl.
syntax error in line 690: '=' expected
Retrieving document from  https://arxiv.org/e-print/2204.02017


/tmp/ipykernel_20966/2881150532.py:48: LatexWarning: 2204.02017 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2209.02722/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


Retrieving document from  https://arxiv.org/e-print/2211.01474


/tmp/ipykernel_20966/2881150532.py:48: LatexWarning: 2211.01474 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2212.00806/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2212.09675/OH119z6QS

Found 84 bibliographic references in tmp_2212.09675/OH119z6QSOsOutflows.bbl.


/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2212.11177/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2301.08310/ges.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:389: LatexWarning: Lat

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [ ]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/reprocessing-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Export documents

We now write the .md files and export relevant images

In [ ]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if not fname:
            # not always 3 figures
            continue
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [ ]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')